Необходимо предложить решение следующей задачи
Пусть в переменной assigneeList' лежит список уникальных идентификаторов исполнителей. Идентификаторы в этом списке не повторяются. 
Суть задания состоит в том, чтобы перераспределить задачи исполнителей, 
	которые находятся дольше days дней в статусах 'Open', 'On support side', 'Verifying' между исполнителями так, 
	чтобы их нагрузка стала одинаковой(или примерно одинаковой с минимальной разницей). 
Под нагрузкой исполнителя понимается количество его задач в статусах 'Open', 'On support side', 'Verifying'.
К переменным из условия можно обратиться в любой части запроса.

Для решения задачи предлагается сделать следующее:

1) Составить запрос, результатом которого будут задачи, требующие распределения в формате [{"key":"id-1", "assignee":"assignee1"}, ...].

2) Составить запрос, результатом которого будет количество задач для каждого исполнителя в формате [{"assignee": "assignee1", "needTasks":5}, ...].

3) Написать небольшой алгоритм, который окончательно распределит задания между исполнителями и решит поставленную задачу.

Формат выходных данных следующий: [{"key": "id-1","assignee": "assignee1"}, ...].
Можно использовать псевдокод или один из перечисленных языков программирования: python, groovy, java, javascript. 
Положить, что результатом выполнения запроса 1 является список ассоциативных массивов(list of map, list of dict), 
	к которому можно обратиться из кода через переменную query1. Аналогично к запросу 2 можно обратиться через переменную query2.

Допускается свой ход решения. Требование к решению: бОльшая часть работы по распределению задач должна лежать на стороне языка запросов.
Итоговый алгоритм не должен сам обрабатывать таблицы, вычленяя из неё нужную информацию. Это задача на умение работать с языком запросов. 
Допускается отсутствие алгоритма вовсе и решение задачи целиком с помощью языка запросов.
К заданию необходимо приложить план решения на естественном языке, 
	который описывает ключевые моменты решения(описывать их подробно не обязательно).

In [1]:
import pandas as pd
import numpy as np

In [2]:
my_train = pd.read_csv(r'./2.st_tasks.csv')

In [3]:
my_train

,key_id,assignee_id,status,created,updated
0,1,1,On support side,2021-01-11 14:57:28,2021-01-12 15:10:16
1,2,3,Verifying,2021-01-11 14:57:28,2021-01-18 15:22:46
2,3,5,Close,2021-01-11 14:57:28,2021-01-21 15:24:42
3,4,7,Open,2021-01-11 14:57:28,2021-01-11 14:57:28
4,5,9,Verifying,2021-01-11 14:57:28,2021-01-18 15:22:46
5,6,1,Open,2021-01-12 14:57:28,2021-01-12 14:57:28
6,7,2,Close,2021-01-12 14:57:28,2021-01-21 15:24:42
7,8,4,On support side,2021-01-12 14:57:28,2021-01-12 15:14:39
8,9,3,On support side,2021-01-12 14:57:28,2021-01-12 15:14:39
9,10,1,Open,2021-01-13 14:57:28,2021-01-13 14:57:28


In [4]:
task_info = pd.DataFrame(np.sort(my_train['assignee_id'].unique()).tolist(), columns=['assignee_id'])
task_info['amount_actual_tasks'] = np.nan
task_info['amount_open_tasks'] = np.nan
task_info['amount_support_tasks'] = np.nan
task_info['amount_verif_tasks'] = np.nan

In [5]:
# Создадим сводную таблицу, с которой далее будет работать(в том числе опеределять нагрузку)
for iter_item in range(1, my_train['assignee_id'].unique().max() + 1):
    amount_actual_tasks = len(my_train.loc[(my_train['assignee_id'] == iter_item) & ((my_train['status'] == 'Open') | 
                                               (my_train['status'] == 'On support side') |
                                               (my_train['status'] == 'Verifying'))  ])
    amount_open_tasks = len(my_train.loc[(my_train['assignee_id'] == iter_item) & (my_train['status'] == 'Open')])
    amount_support_tasks = len(my_train.loc[(my_train['assignee_id'] == iter_item) & (my_train['status'] == 'On support side')])
    amount_verif_tasks = len(my_train.loc[(my_train['assignee_id'] == iter_item) & (my_train['status'] == 'Verifying')])
    
    task_info.loc[task_info['assignee_id'] == int(iter_item), 'amount_actual_tasks'] = int(amount_actual_tasks)
    task_info.loc[task_info['assignee_id'] == int(iter_item), 'amount_open_tasks'] = int(amount_open_tasks)
    task_info.loc[task_info['assignee_id'] == int(iter_item), 'amount_support_tasks'] = int(amount_support_tasks)
    task_info.loc[task_info['assignee_id'] == int(iter_item), 'amount_verif_tasks'] = int(amount_verif_tasks)


In [6]:
task_info

,assignee_id,amount_actual_tasks,amount_open_tasks,amount_support_tasks,amount_verif_tasks
0,1,7.0,5.0,2.0,0.0
1,2,4.0,2.0,2.0,0.0
2,3,3.0,1.0,1.0,1.0
3,4,1.0,0.0,1.0,0.0
4,5,2.0,1.0,0.0,1.0
5,6,2.0,0.0,2.0,0.0
6,7,3.0,1.0,1.0,1.0
7,8,2.0,1.0,1.0,0.0
8,9,5.0,3.0,1.0,1.0
9,10,1.0,0.0,1.0,0.0


### 1) Составить запрос, результатом которого будут задачи, требующие распределения в формате 
[{"key":"id-1", "assignee":"assignee1"}, ...]

In [7]:
# Напишем алгоритм, который сначала выберет тех сотрудников(assignee), у которых общее кол-во задач превышает среднее значение(по таблице task_info['amount_actual_tasks']),
#  а затем, случайно выберет необходимое кол-во задач сотрудника где статус Open, которые нужно перераспределить. 
need_to_reassign = []
list_excess_avg = np.sort(task_info.loc[task_info['amount_actual_tasks'] > task_info['amount_actual_tasks'].mean()]['assignee_id'].astype('int')).tolist()
for itr in list_excess_avg:
    list_to_reassign = my_train.loc[(my_train['assignee_id'] == itr) & 
             (my_train['status'] == 'Open')].sample(n = int(task_info.loc[task_info['assignee_id'] == itr]['amount_actual_tasks']) -
                                                    int(task_info['amount_actual_tasks'].mean()))
    
    for key, assignee in zip(list_to_reassign['key_id'], list_to_reassign['assignee_id']):
        loc_dict = {}
        loc_dict['key'] = key
        loc_dict['assignee'] = assignee
        need_to_reassign.append(loc_dict)

In [8]:
need_to_reassign

[{'key': 10, 'assignee': 1},
 {'key': 6, 'assignee': 1},
 {'key': 27, 'assignee': 1},
 {'key': 19, 'assignee': 1},
 {'key': 20, 'assignee': 2},
 {'key': 14, 'assignee': 9},
 {'key': 28, 'assignee': 9}]

###  2) Составить запрос, результатом которого будет количество задач для каждого исполнителя в формате [{"assignee": "assignee1", "needTasks":5}, ...].

In [9]:
# Выберем всех, у кого нагрузка меньше среднего
need_tasks = []
list_excess_avg = np.sort(task_info.loc[task_info['amount_actual_tasks'] < task_info['amount_actual_tasks'].mean()]['assignee_id'].astype('int')).tolist()
for itr in list_excess_avg:
    add_task = int(task_info['amount_actual_tasks'].mean()) - int(task_info.loc[task_info['assignee_id'] == itr]['amount_actual_tasks'])
    
    loc_dict = {}
    loc_dict['assignee'] = itr
    loc_dict['needTasks'] = add_task
    need_tasks.append(loc_dict)


In [10]:
need_tasks

[{'assignee': 4, 'needTasks': 2},
 {'assignee': 5, 'needTasks': 1},
 {'assignee': 6, 'needTasks': 1},
 {'assignee': 8, 'needTasks': 1},
 {'assignee': 10, 'needTasks': 2}]

### 3) Написать небольшой алгоритм, который окончательно распределит задания между исполнителями и решит поставленную задачу.

In [11]:
# Возьмем 2 предыдущих алгоритма(2 листа: need_to_reassign, need_tasks) и построим циклы.
# Если работники не дозагружены, но при этом нет тех, кто перегружен, алгоритм не будет производить действия.
cnt = 0
len_list_re = len(need_to_reassign)

if len(need_tasks) > 0:
    for iterr in need_tasks:
        val1 = iterr.get('assignee')
        val2 = iterr.get('needTasks')

        if len(need_to_reassign) > 0:
            for _ in range(1, val2 + 1):

                if (len_list_re - cnt) > 0:
                    val3 = need_to_reassign[cnt].get('key')
                    val4 = need_to_reassign[cnt].get('assignee')

                    my_train.loc[my_train['key_id'] == val3, 'assignee_id'] = val1
                    print(f"key_id со значением {val3}: assignee = {val4} заменен на {val1}")
                    cnt += 1
                

key_id со значением 10: assignee = 1 заменен на 4
key_id со значением 6: assignee = 1 заменен на 4
key_id со значением 27: assignee = 1 заменен на 5
key_id со значением 19: assignee = 1 заменен на 6
key_id со значением 20: assignee = 2 заменен на 8
key_id со значением 14: assignee = 9 заменен на 10
key_id со значением 28: assignee = 9 заменен на 10


In [12]:
my_train

,key_id,assignee_id,status,created,updated
0,1,1,On support side,2021-01-11 14:57:28,2021-01-12 15:10:16
1,2,3,Verifying,2021-01-11 14:57:28,2021-01-18 15:22:46
2,3,5,Close,2021-01-11 14:57:28,2021-01-21 15:24:42
3,4,7,Open,2021-01-11 14:57:28,2021-01-11 14:57:28
4,5,9,Verifying,2021-01-11 14:57:28,2021-01-18 15:22:46
5,6,4,Open,2021-01-12 14:57:28,2021-01-12 14:57:28
6,7,2,Close,2021-01-12 14:57:28,2021-01-21 15:24:42
7,8,4,On support side,2021-01-12 14:57:28,2021-01-12 15:14:39
8,9,3,On support side,2021-01-12 14:57:28,2021-01-12 15:14:39
9,10,4,Open,2021-01-13 14:57:28,2021-01-13 14:57:28
